In [1]:
import numpy as np
import pandas as pd
import re
from scipy.stats import zscore

/Users/sukaraca/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Processing

In [2]:
df = pd.read_csv("liwc_test_prob.csv")

In [3]:
df.columns

Index(['article_index', 'outlet', 'original_text', 'processed_text',
       'probability_0', 'probability_1', 'extremity_0', 'extremity_1', 'label',
       'Segment', 'WC', 'WPS', 'number', 'quantity', 'Drives', 'affiliation',
       'achieve', 'power', 'Cognition', 'cogproc', 'insight', 'cause',
       'discrep', 'tentat', 'certitude', 'differ', 'emo_pos', 'emo_neg',
       'emo_anx', 'emo_anger', 'emo_sad', 'Social', 'prosocial', 'polite',
       'socrefs', 'family', 'friend', 'female', 'male', 'politic', 'ethnicity',
       'Lifestyle', 'leisure', 'home', 'work', 'money', 'relig', 'Physical',
       'health', 'sexual', 'food', 'death', 'need', 'want', 'acquire', 'lack',
       'fulfill', 'fatigue', 'reward', 'risk', 'Perception', 'motion', 'space',
       'visual', 'auditory', 'feeling', 'time', 'focuspast', 'focuspresent',
       'focusfuture'],
      dtype='object')

### As there are some correlated categories (subcategories and main categories co-existing, I drop them)

In [4]:
df.drop(['article_index', 'Segment', 'extremity_1',
         'WC', 'WPS', 'number', 'quantity', 'affiliation',
         'achieve', 'power', 'cogproc', 'insight', 'cause',
         'discrep', 'tentat', 'certitude', 'differ', 'prosocial',
         'polite', 'socrefs', 'family', 'friend', 'female',
         'male', 'politic', 'ethnicity', 'leisure', 'home',
         'work', 'money', 'relig', 'health', 'sexual', 'food',
         'death', 'need', 'want', 'acquire', 'lack', 'fulfill',
         'fatigue', 'reward', 'risk', 'Perception', 'motion',
         'space', 'visual', 'auditory', 'feeling', 'time',
        ], axis=1, inplace=True)

In [5]:
df.rename(columns={'probability_0':'Probability_con',
                   'probability_1':'Probability_lib',
                   'extremity_0':'Extremity'}, inplace=True)

In [6]:
df.head(5)

,outlet,original_text,processed_text,Probability_con,Probability_lib,Extremity,label,Drives,Cognition,emo_pos,emo_neg,emo_anx,emo_anger,emo_sad,Social,Lifestyle,Physical,focuspast,focuspresent,focusfuture
0,Daily Caller,Journalists were quick to take to Twitter and ...,journalists were quick to take to twitter and ...,0.999892,0.000108,0.499892,0,4.90,8.50,0.0,0.65,0.00,0.33,0.00,13.73,3.92,0.65,4.90,1.96,0.33
1,NYTimes,"In June, Josue, a 21-year-old Honduran, reache...","in june, josue, a 21 - year - old honduran, re...",0.000023,0.999977,0.499977,1,3.10,7.40,0.0,0.24,0.00,0.00,0.24,6.68,3.34,0.24,3.10,2.63,0.48
2,HuffPost,A Massachusetts county sheriff has proposed se...,a massachusetts county sheriff has proposed se...,0.000039,0.999961,0.499961,1,10.15,10.15,0.0,0.00,0.00,0.00,0.00,15.59,6.19,0.00,2.72,2.72,1.73
3,FoxNews,We’ve got a question for all these people clai...,we ’ ve got a question for all these people cl...,0.999971,0.000029,0.499971,0,7.02,11.14,0.0,0.73,0.24,0.00,0.24,14.04,1.21,0.48,4.60,2.91,0.24
4,CNN,President Donald Trump won the White House in ...,president donald trump won the white house in ...,0.000028,0.999972,0.499972,1,7.02,9.93,0.0,0.73,0.48,0.24,0.00,14.29,2.18,0.73,3.39,5.33,0.48


In [7]:
df.dtypes[df.dtypes != 'float64']

outlet            object
original_text     object
processed_text    object
label              int64
dtype: object

In [8]:
df["label"] = df["label"].astype('category')

## Adding moral-emotional Dics

In [9]:
affect_dict_path = 'only_affect_dict.txt'
with open(affect_dict_path, 'r') as file:
    affect_dict = file.readlines()
    # Remove newline characters
    affect_dict = [line.strip() for line in affect_dict]
    
affect_dict = [root.rstrip('*') for root in affect_dict]

In [10]:
moral_dict_path = 'only_moral_dict.txt'
with open(moral_dict_path, 'r') as file:
    moral_dict = file.readlines()
    # Remove newline characters
    moral_dict = [line.strip() for line in moral_dict]
    
moral_dict = [root.rstrip('*') for root in moral_dict]

In [11]:
me_dict_path = 'moral_emotional_dict.txt'
with open(me_dict_path, 'r') as file:
    me_dict = file.readlines()
    # Remove newline characters
    me_dict = [line.strip() for line in me_dict]
    
me_dict = [root.rstrip('*') for root in me_dict]

In [12]:
# Define the function to calculate percentages of words
def calculate_word_percentage(text, word_roots):
    text = text.lower()
    words = re.findall(r'\b\w+\b', text)
    found_words = [word for word in words if any(root in word for root in word_roots)]
    percentage = (len(found_words) / len(words)) * 100
    return percentage

# Apply the function to the DataFrame
df['Moral'] = df['processed_text'].apply(lambda x: calculate_word_percentage(x, moral_dict))
df['Affect'] = df['processed_text'].apply(lambda x: calculate_word_percentage(x, affect_dict))
df['Moral-Emotional'] = df['processed_text'].apply(lambda x: calculate_word_percentage(x, me_dict))

In [13]:
df.head(5)

,outlet,original_text,processed_text,Probability_con,Probability_lib,Extremity,label,Drives,Cognition,emo_pos,...,emo_sad,Social,Lifestyle,Physical,focuspast,focuspresent,focusfuture,Moral,Affect,Moral-Emotional
0,Daily Caller,Journalists were quick to take to Twitter and ...,journalists were quick to take to twitter and ...,0.999892,0.000108,0.499892,0,4.90,8.50,0.0,...,0.00,13.73,3.92,0.65,4.90,1.96,0.33,5.882353,5.882353,1.633987
1,NYTimes,"In June, Josue, a 21-year-old Honduran, reache...","in june, josue, a 21 - year - old honduran, re...",0.000023,0.999977,0.499977,1,3.10,7.40,0.0,...,0.24,6.68,3.34,0.24,3.10,2.63,0.48,5.489260,9.785203,2.386635
2,HuffPost,A Massachusetts county sheriff has proposed se...,a massachusetts county sheriff has proposed se...,0.000039,0.999961,0.499961,1,10.15,10.15,0.0,...,0.00,15.59,6.19,0.00,2.72,2.72,1.73,5.693069,8.168317,0.990099
3,FoxNews,We’ve got a question for all these people clai...,we ’ ve got a question for all these people cl...,0.999971,0.000029,0.499971,0,7.02,11.14,0.0,...,0.24,14.04,1.21,0.48,4.60,2.91,0.24,4.842615,10.411622,2.179177
4,CNN,President Donald Trump won the White House in ...,president donald trump won the white house in ...,0.000028,0.999972,0.499972,1,7.02,9.93,0.0,...,0.00,14.29,2.18,0.73,3.39,5.33,0.48,4.358354,12.590799,0.726392


In [14]:
df.describe()

,Probability_con,Probability_lib,Extremity,Drives,Cognition,emo_pos,emo_neg,emo_anx,emo_anger,emo_sad,Social,Lifestyle,Physical,focuspast,focuspresent,focusfuture,Moral,Affect,Moral-Emotional
count,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000
mean,0.393879,0.606121,0.489173,7.473222,8.487111,0.149000,0.409444,0.121333,0.152444,0.034833,12.749611,4.587556,0.915556,4.361111,2.731111,0.921556,5.834889,9.140689,1.321941
std,0.481642,0.481642,0.051736,2.706096,2.321491,0.295081,0.450487,0.262864,0.274519,0.134154,3.831701,2.241594,1.222299,1.849276,1.366588,0.792667,2.039960,2.064908,0.871181
min,0.000015,0.000018,0.093808,2.400000,2.770000,0.000000,0.000000,0.000000,0.000000,0.000000,2.880000,0.640000,0.000000,0.650000,0.000000,0.000000,0.925926,3.952569,0.000000
25%,0.000023,0.000139,0.499849,5.542500,7.002500,0.000000,0.000000,0.000000,0.000000,0.000000,10.132500,2.837500,0.240000,3.022500,1.830000,0.305000,4.502077,7.881837,0.717752
50%,0.000161,0.999839,0.499969,7.395000,8.400000,0.000000,0.260000,0.000000,0.000000,0.000000,12.455000,4.100000,0.495000,4.160000,2.560000,0.740000,5.656047,9.339564,1.182040
75%,0.999861,0.999977,0.499979,9.152500,9.977500,0.240000,0.597500,0.230000,0.240000,0.000000,15.092500,6.065000,1.192500,5.295000,3.382500,1.372500,7.238235,10.538642,1.868248
max,0.999982,0.999985,0.499985,16.360000,14.840000,2.550000,2.230000,1.520000,2.110000,1.170000,27.270000,12.740000,8.020000,10.780000,7.020000,4.480000,12.144703,14.713217,4.450262


## Applying Z-Scores

In [15]:
selected_columns = df.iloc[:, 7:23]

In [16]:
z_scores = selected_columns.apply(zscore)

In [17]:
# Add the z-scores back to the DataFrame
for i, col in enumerate(selected_columns.columns):
    df[f'{col}_zscore'] = z_scores.iloc[:, i]

In [18]:
df.columns[:39]

Index(['outlet', 'original_text', 'processed_text', 'Probability_con',
       'Probability_lib', 'Extremity', 'label', 'Drives', 'Cognition',
       'emo_pos', 'emo_neg', 'emo_anx', 'emo_anger', 'emo_sad', 'Social',
       'Lifestyle', 'Physical', 'focuspast', 'focuspresent', 'focusfuture',
       'Moral', 'Affect', 'Moral-Emotional', 'Drives_zscore',
       'Cognition_zscore', 'emo_pos_zscore', 'emo_neg_zscore',
       'emo_anx_zscore', 'emo_anger_zscore', 'emo_sad_zscore', 'Social_zscore',
       'Lifestyle_zscore', 'Physical_zscore', 'focuspast_zscore',
       'focuspresent_zscore', 'focusfuture_zscore', 'Moral_zscore',
       'Affect_zscore', 'Moral-Emotional_zscore'],
      dtype='object')

In [19]:
columns_to_drop = ['original_text', 'processed_text' ,
                   'Probability_con', 'Probability_lib',
                   'Drives', 'Cognition','emo_pos',
                   'emo_neg', 'emo_anx', 'emo_anger', 'emo_sad',
                   'Social', 'Lifestyle', 'Physical', 'focuspast',
                   'focuspresent', 'focusfuture', 'Moral', 'Affect',
                   'Moral-Emotional',]

In [24]:
columns_to_drop_2 = ['original_text', 'processed_text' ,
                   'Extremity', 'Probability_lib',
                   'Drives', 'Cognition','emo_pos',
                   'emo_neg', 'emo_anx', 'emo_anger', 'emo_sad',
                   'Social', 'Lifestyle', 'Physical', 'focuspast',
                   'focuspresent', 'focusfuture', 'Moral', 'Affect',
                   'Moral-Emotional',]

In [20]:
df_z = df.drop(columns=columns_to_drop)

In [21]:
df_z

,outlet,Extremity,label,Drives_zscore,Cognition_zscore,emo_pos_zscore,emo_neg_zscore,emo_anx_zscore,emo_anger_zscore,emo_sad_zscore,Social_zscore,Lifestyle_zscore,Physical_zscore,focuspast_zscore,focuspresent_zscore,focusfuture_zscore,Moral_zscore,Affect_zscore,Moral-Emotional_zscore
0,Daily Caller,0.499892,0,-0.953551,0.005567,-0.506355,0.535480,-0.462869,0.648593,-0.260377,0.256576,-0.298635,-0.217865,0.292218,-0.565834,-0.748367,0.023332,-1.582358,0.359187
1,NYTimes,0.499977,1,-1.620571,-0.469588,-0.506355,-0.377186,-0.462869,-0.556865,1.533608,-1.588470,-0.558101,-0.554234,-0.683851,-0.074194,-0.558605,-0.169902,0.312998,1.225537
2,HuffPost,0.499961,1,0.991925,0.718300,-0.506355,-0.911429,-0.462869,-0.556865,-0.260377,0.743354,0.716862,-0.751133,-0.889910,-0.008153,1.022750,-0.069715,-0.472217,-0.381973
3,FoxNews,0.499971,0,-0.167949,1.145940,-0.506355,0.713561,0.452696,-0.556865,1.533608,0.337706,-1.510968,-0.357335,0.129540,0.131267,-0.862225,-0.487775,0.617208,0.986738
4,CNN,0.499972,1,-0.167949,0.623269,-0.506355,0.713561,1.368261,0.319832,-0.260377,0.403133,-1.077033,-0.152232,-0.526595,1.907039,-0.558605,-0.725825,1.675490,-0.685518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,HuffPost,0.499933,1,-0.986902,0.683744,0.275267,0.112537,0.414548,-0.556865,-0.260377,-0.101965,0.609497,-0.562438,-0.504905,0.512838,0.567320,0.056417,1.450810,-0.204633
176,HuffPost,0.495781,1,0.006217,-0.987939,2.144361,-0.332665,-0.462869,-0.556865,1.683107,-0.630616,-0.781779,-0.111211,-0.542863,-0.485117,-0.836923,3.101734,-0.799911,3.534732
177,HuffPost,0.499833,1,-1.335235,0.830610,0.309250,-0.377186,-0.462869,0.319832,-0.260377,1.224898,-0.535733,0.438465,2.493796,-1.651844,-0.862225,-1.916075,1.557903,1.822866
178,HuffPost,0.499985,1,0.954868,-1.186640,-0.506355,1.225544,-0.462869,0.319832,1.533608,0.418836,0.515552,2.391047,-1.584003,1.679565,-0.862225,0.894964,0.324366,0.956734


In [22]:
df_z.to_csv('z_score_extremity.csv', index=False) 

In [23]:
df_z.describe()

,Extremity,Drives_zscore,Cognition_zscore,emo_pos_zscore,emo_neg_zscore,emo_anx_zscore,emo_anger_zscore,emo_sad_zscore,Social_zscore,Lifestyle_zscore,Physical_zscore,focuspast_zscore,focuspresent_zscore,focusfuture_zscore,Moral_zscore,Affect_zscore,Moral-Emotional_zscore
count,180.000000,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02
mean,0.489173,-1.529641e-16,4.194176e-16,-6.507141e-17,1.776357e-16,7.401487e-17,-6.167906e-19,-5.952029e-17,-4.903485e-16,6.106227e-17,-2.837237e-17,2.238950e-16,-1.517305e-16,1.541976e-17,5.181041e-16,-1.279840e-16,-5.982869e-17
std,0.051736,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00
min,0.093808,-1.879968e+00,-2.469559e+00,-5.063548e-01,-9.114289e-01,-4.628691e-01,-5.568645e-01,-2.603770e-01,-2.582963e+00,-1.765961e+00,-7.511333e-01,-2.012389e+00,-2.004063e+00,-1.165845e+00,-2.413114e+00,-2.519527e+00,-1.521646e+00
25%,0.499849,-7.154617e-01,-6.412915e-01,-5.063548e-01,-9.114289e-01,-4.628691e-01,-5.568645e-01,-2.603770e-01,-6.849207e-01,-7.828970e-01,-5.542343e-01,-7.258759e-01,-6.612266e-01,-7.799942e-01,-6.551748e-01,-6.113412e-01,-6.954633e-01
50%,0.499969,-2.898656e-02,-3.762845e-02,-5.063548e-01,-3.326654e-01,-4.628691e-01,-5.568645e-01,-2.603770e-01,-7.710229e-02,-2.181107e-01,-3.450291e-01,-1.090546e-01,-1.255597e-01,-2.296829e-01,-8.791395e-02,9.658036e-02,-1.610360e-01
75%,0.499979,6.222847e-01,6.437873e-01,3.092502e-01,4.186142e-01,4.145476e-01,3.198318e-01,-2.603770e-01,6.131544e-01,6.609429e-01,2.272087e-01,5.064111e-01,4.779829e-01,5.704823e-01,6.898470e-01,6.788931e-01,6.288377e-01
max,0.499985,3.293145e+00,2.744189e+00,8.159448e+00,4.052581e+00,5.335711e+00,7.150757e+00,8.485299e+00,3.800112e+00,3.647041e+00,5.828575e+00,3.480710e+00,3.147145e+00,4.501729e+00,3.101734e+00,2.706208e+00,3.600915e+00


In [30]:
df_z[df_z['label']==1].describe()

,Extremity,Drives_zscore,Cognition_zscore,emo_pos_zscore,emo_neg_zscore,emo_anx_zscore,emo_anger_zscore,emo_sad_zscore,Social_zscore,Lifestyle_zscore,Physical_zscore,focuspast_zscore,focuspresent_zscore,focusfuture_zscore,Moral_zscore,Affect_zscore,Moral-Emotional_zscore
count,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000
mean,0.497169,0.071108,0.060570,-0.060038,0.069501,-0.029671,0.038558,0.058554,-0.024673,0.227008,-0.085232,0.108693,-0.059845,-0.142533,-0.129041,0.101242,-0.025460
std,0.020007,0.866006,0.972797,0.820399,1.077493,0.953093,1.151749,1.172601,1.082022,1.090860,0.823366,1.075139,0.963686,0.860935,1.079243,0.968776,0.927643
min,0.313609,-1.861440,-2.175827,-0.506355,-0.911429,-0.462869,-0.556865,-0.260377,-2.582963,-1.408076,-0.751133,-2.012389,-1.651844,-1.165845,-2.166027,-2.063645,-1.521646
25%,0.499967,-0.453286,-0.531142,-0.506355,-0.911429,-0.462869,-0.556865,-0.260377,-0.849143,-0.630796,-0.703960,-0.655382,-0.672233,-0.846411,-0.789983,-0.594880,-0.685011
50%,0.499976,0.169267,-0.050587,-0.506355,-0.366056,-0.462869,-0.556865,-0.260377,-0.007749,0.135300,-0.365539,-0.057540,-0.213615,-0.463723,-0.163025,0.247300,-0.172604
75%,0.499981,0.656562,0.683744,0.309250,0.374094,-0.462869,0.319832,-0.260377,0.609229,0.770545,0.227209,0.763985,0.360576,0.399696,0.572373,0.772156,0.487851
max,0.499985,3.293145,2.744189,2.790049,4.052581,4.191255,7.150757,8.485299,3.174628,3.647041,2.989948,3.480710,2.802264,2.819168,3.101734,2.539288,3.534732


In [31]:
df_z[df_z['label']==0].describe()

,Extremity,Drives_zscore,Cognition_zscore,emo_pos_zscore,emo_neg_zscore,emo_anx_zscore,emo_anger_zscore,emo_sad_zscore,Social_zscore,Lifestyle_zscore,Physical_zscore,focuspast_zscore,focuspresent_zscore,focusfuture_zscore,Moral_zscore,Affect_zscore,Moral-Emotional_zscore
count,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000
mean,0.481178,-0.071108,-0.060570,0.060038,-0.069501,0.029671,-0.038558,-0.058554,0.024673,-0.227008,0.085232,-0.108693,0.059845,0.142533,0.129041,-0.101242,0.025460
std,0.069669,1.123512,1.033788,1.158502,0.922882,1.054659,0.832430,0.800340,0.922209,0.853389,1.153191,0.917962,1.042371,1.113632,0.907760,1.031123,1.077330
min,0.093808,-1.879968,-2.469559,-0.506355,-0.911429,-0.462869,-0.556865,-0.260377,-1.944394,-1.765961,-0.751133,-1.562313,-2.004063,-1.165845,-2.413114,-2.519527,-1.521646
25%,0.499597,-0.953551,-0.770879,-0.506355,-0.911429,-0.462869,-0.556865,-0.260377,-0.636505,-0.882434,-0.546030,-0.790947,-0.646551,-0.748367,-0.410363,-0.623082,-0.718522
50%,0.499942,-0.156832,-0.028989,-0.506355,-0.332665,-0.462869,-0.556865,-0.260377,-0.146455,-0.354554,-0.230171,-0.277155,-0.074194,-0.027270,0.009295,-0.100018,-0.129196
75%,0.499972,0.466647,0.575754,0.343234,0.418614,0.443159,0.356361,-0.260377,0.588946,0.331018,0.223107,0.444051,0.619237,0.715967,0.809701,0.526755,0.745078
max,0.499982,2.863288,2.588684,8.159448,3.050875,5.335711,2.182811,5.345826,3.800112,1.893407,5.828575,2.721545,3.147145,4.501729,2.024159,2.706208,3.600915


In [25]:
df_p = df.drop(columns=columns_to_drop_2)

In [26]:
df_p

,outlet,Probability_con,label,Drives_zscore,Cognition_zscore,emo_pos_zscore,emo_neg_zscore,emo_anx_zscore,emo_anger_zscore,emo_sad_zscore,Social_zscore,Lifestyle_zscore,Physical_zscore,focuspast_zscore,focuspresent_zscore,focusfuture_zscore,Moral_zscore,Affect_zscore,Moral-Emotional_zscore
0,Daily Caller,0.999892,0,-0.953551,0.005567,-0.506355,0.535480,-0.462869,0.648593,-0.260377,0.256576,-0.298635,-0.217865,0.292218,-0.565834,-0.748367,0.023332,-1.582358,0.359187
1,NYTimes,0.000023,1,-1.620571,-0.469588,-0.506355,-0.377186,-0.462869,-0.556865,1.533608,-1.588470,-0.558101,-0.554234,-0.683851,-0.074194,-0.558605,-0.169902,0.312998,1.225537
2,HuffPost,0.000039,1,0.991925,0.718300,-0.506355,-0.911429,-0.462869,-0.556865,-0.260377,0.743354,0.716862,-0.751133,-0.889910,-0.008153,1.022750,-0.069715,-0.472217,-0.381973
3,FoxNews,0.999971,0,-0.167949,1.145940,-0.506355,0.713561,0.452696,-0.556865,1.533608,0.337706,-1.510968,-0.357335,0.129540,0.131267,-0.862225,-0.487775,0.617208,0.986738
4,CNN,0.000028,1,-0.167949,0.623269,-0.506355,0.713561,1.368261,0.319832,-0.260377,0.403133,-1.077033,-0.152232,-0.526595,1.907039,-0.558605,-0.725825,1.675490,-0.685518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,HuffPost,0.000067,1,-0.986902,0.683744,0.275267,0.112537,0.414548,-0.556865,-0.260377,-0.101965,0.609497,-0.562438,-0.504905,0.512838,0.567320,0.056417,1.450810,-0.204633
176,HuffPost,0.004219,1,0.006217,-0.987939,2.144361,-0.332665,-0.462869,-0.556865,1.683107,-0.630616,-0.781779,-0.111211,-0.542863,-0.485117,-0.836923,3.101734,-0.799911,3.534732
177,HuffPost,0.000167,1,-1.335235,0.830610,0.309250,-0.377186,-0.462869,0.319832,-0.260377,1.224898,-0.535733,0.438465,2.493796,-1.651844,-0.862225,-1.916075,1.557903,1.822866
178,HuffPost,0.000015,1,0.954868,-1.186640,-0.506355,1.225544,-0.462869,0.319832,1.533608,0.418836,0.515552,2.391047,-1.584003,1.679565,-0.862225,0.894964,0.324366,0.956734


In [27]:
df_p.to_csv('z_score_prob.csv', index=False) 

In [29]:
df_p.describe()

,Probability_con,Drives_zscore,Cognition_zscore,emo_pos_zscore,emo_neg_zscore,emo_anx_zscore,emo_anger_zscore,emo_sad_zscore,Social_zscore,Lifestyle_zscore,Physical_zscore,focuspast_zscore,focuspresent_zscore,focusfuture_zscore,Moral_zscore,Affect_zscore,Moral-Emotional_zscore
count,180.000000,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02
mean,0.393879,-1.529641e-16,4.194176e-16,-6.507141e-17,1.776357e-16,7.401487e-17,-6.167906e-19,-5.952029e-17,-4.903485e-16,6.106227e-17,-2.837237e-17,2.238950e-16,-1.517305e-16,1.541976e-17,5.181041e-16,-1.279840e-16,-5.982869e-17
std,0.481642,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00,1.002789e+00
min,0.000015,-1.879968e+00,-2.469559e+00,-5.063548e-01,-9.114289e-01,-4.628691e-01,-5.568645e-01,-2.603770e-01,-2.582963e+00,-1.765961e+00,-7.511333e-01,-2.012389e+00,-2.004063e+00,-1.165845e+00,-2.413114e+00,-2.519527e+00,-1.521646e+00
25%,0.000023,-7.154617e-01,-6.412915e-01,-5.063548e-01,-9.114289e-01,-4.628691e-01,-5.568645e-01,-2.603770e-01,-6.849207e-01,-7.828970e-01,-5.542343e-01,-7.258759e-01,-6.612266e-01,-7.799942e-01,-6.551748e-01,-6.113412e-01,-6.954633e-01
50%,0.000161,-2.898656e-02,-3.762845e-02,-5.063548e-01,-3.326654e-01,-4.628691e-01,-5.568645e-01,-2.603770e-01,-7.710229e-02,-2.181107e-01,-3.450291e-01,-1.090546e-01,-1.255597e-01,-2.296829e-01,-8.791395e-02,9.658036e-02,-1.610360e-01
75%,0.999861,6.222847e-01,6.437873e-01,3.092502e-01,4.186142e-01,4.145476e-01,3.198318e-01,-2.603770e-01,6.131544e-01,6.609429e-01,2.272087e-01,5.064111e-01,4.779829e-01,5.704823e-01,6.898470e-01,6.788931e-01,6.288377e-01
max,0.999982,3.293145e+00,2.744189e+00,8.159448e+00,4.052581e+00,5.335711e+00,7.150757e+00,8.485299e+00,3.800112e+00,3.647041e+00,5.828575e+00,3.480710e+00,3.147145e+00,4.501729e+00,3.101734e+00,2.706208e+00,3.600915e+00
